In [1]:
!pip install datasets
!pip install --upgrade transformers

In [2]:
import datasets

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

2024-04-12 15:06:01.411615: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 15:06:01.411724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 15:06:01.706480: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
data=pd.read_csv("/kaggle/input/zindi-vaccinate-or-not-v2/Train (1).csv")

In [5]:
data.head(5)

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [6]:
data2=data.drop("agreement",axis=1)

In [7]:
data2.dropna(inplace=True)

In [8]:
data2.label.value_counts()

label
 0.000000    4908
 1.000000    4053
-1.000000    1038
 0.666667       1
Name: count, dtype: int64

In [9]:
data2.loc[(data2.label>0.66) & (data2.label<1)]

,tweet_id,safe_text,label
4799,I cannot believe in this day and age some pare...,1,0.666667


In [10]:
data2.drop(index=4799,inplace=True)

In [11]:
dict2={-1:0,0:1,1:2}

In [12]:
data2.label=data2.label.map(dict2)

In [13]:
from sklearn.model_selection import train_test_split
train_data,val_data=train_test_split(data2,train_size=0.8,stratify=data2["label"])

In [14]:
train_data.head(5)

,tweet_id,safe_text,label
7373,787KX4ZT,Chicago starting outreach to promote meningiti...,2
2135,78NRP6VB,If you get the flu shot for you or your child ...,0
8096,5ISNLRUX,160 cases of measles in the US?!?!?! Unless th...,2
1677,87DFM49R,"Exotic vaccinations, oh my! (@ Waco Infectious...",2
5310,U9NO969S,<user> <user> <user> my uncle who got a brain ...,1


In [15]:
from datasets import Dataset,DatasetDict
tds = Dataset.from_pandas(train_data)
vds = Dataset.from_pandas(val_data)
ds = DatasetDict()
ds['train'] = tds
ds['validation'] = vds

In [16]:
ds

DatasetDict({
    train: Dataset({
        features: ['tweet_id', 'safe_text', 'label', '__index_level_0__'],
        num_rows: 7999
    })
    validation: Dataset({
        features: ['tweet_id', 'safe_text', 'label', '__index_level_0__'],
        num_rows: 2000
    })
})

In [17]:
ds["train"].features

{'tweet_id': Value(dtype='string', id=None),
 'safe_text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [18]:
from transformers import AutoTokenizer

checkpoint = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(example):
  return tokenizer(example["safe_text"], truncation=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [19]:
tokenized_datasets = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['tweet_id', 'safe_text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7999
    })
    validation: Dataset({
        features: ['tweet_id', 'safe_text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [21]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer,return_tensors="tf")

In [22]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(columns=["attention_mask", "input_ids",'token_type_ids'],label_cols=["label"],shuffle=True,collate_fn=data_collator,batch_size=8)
tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(columns=["attention_mask", "input_ids",'token_type_ids'],label_cols=["label"],shuffle=False,collate_fn=data_collator,batch_size=8,)

/opt/conda/lib/python3.10/site-packages/datasets/arrow_dataset.py:401: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [23]:
id2label = {0: "NEGATIVE",1:"NEUTRAL", 2: "POSITIVE"}
label2id = {"NEGATIVE": 0,"NEUTRAL":1, "POSITIVE": 2}
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=3,id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# learning rate and other parameters
batch_size = 8
num_epochs = 4
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=5e-6, end_learning_rate=0.0, decay_steps=num_train_steps)

In [25]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
callbacks=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=20,restore_best_weights=True)
model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=lr_scheduler), metrics=["accuracy"])

In [26]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=4)

Epoch 1/4
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1712934486.629428     103 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1000/1000 [==============================] - 254s 148ms/step - loss: 0.7275 - accuracy: 0.7005 - val_loss: 0.6370 - val_accuracy: 0.7410
Epoch 2/4
1000/1000 [==============================] - 125s 125ms/step - loss: 0.5907 - accuracy: 0.7662 - val_loss: 0.6008 - val_accuracy: 0.7525
Epoch 3/4
1000/1000 [==============================] - 125s 125ms/step - loss: 0.5056 - accuracy: 0.8065 - val_loss: 0.5763 - val_accuracy: 0.7675
Epoch 4/4
1000/1000 [==============================] - 125s 125ms/step - loss: 0.4546 - accuracy: 0.8297 - val_loss: 0.5822 - val_accuracy: 0.7625


In [35]:
val_pred=model.predict(tf_validation_dataset)

250/250 [==============================] - 9s 35ms/step


In [36]:
val_pred[0]

array([[-2.0413592 ,  1.2716659 ,  0.7896246 ],
       [-1.7864751 ,  2.5816245 , -0.46966088],
       [ 0.1193995 , -1.2449514 ,  0.8398983 ],
       ...,
       [-2.1956196 ,  2.9457924 , -0.6188174 ],
       [-1.2064959 , -0.08045156,  0.8860421 ],
       [-1.1632723 , -0.7027353 ,  1.5587939 ]], dtype=float32)

In [37]:
len(val_pred[0])

2000

In [38]:
softmax = tf.nn.softmax(val_pred[0])

In [39]:
softmax

<tf.Tensor: shape=(2000, 3), dtype=float32, numpy=
array([[0.02201178, 0.60462147, 0.37336674],
       [0.01195814, 0.94341993, 0.04462197],
       [0.30202314, 0.07718096, 0.6207959 ],
       ...,
       [0.00565622, 0.9669707 , 0.02737303],
       [0.08204189, 0.25297084, 0.66498727],
       [0.05619037, 0.08905751, 0.8547521 ]], dtype=float32)>

In [40]:
pred=[]
for i in range(0,len(val_pred[0])):
    predicted_class_id = int(tf.math.argmax(softmax, axis=-1)[i])
    pred.append(predicted_class_id)

In [41]:
from sklearn.metrics import classification_report
print(classification_report(val_data["label"],pred))

              precision    recall  f1-score   support

           0       0.56      0.38      0.45       207
           1       0.87      0.78      0.82       982
           2       0.69      0.84      0.76       811

    accuracy                           0.76      2000
   macro avg       0.71      0.67      0.68      2000
weighted avg       0.77      0.76      0.76      2000

